In [2]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from NewOrleans_airbnb import NewOrleans_airbnb
import datetime as dt
from config import api_key

importing Jupyter notebook from NewOrleans_airbnb.ipynb


NewOrleans_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
NewOrleans_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
NewOrleans_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [4]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("New Orleans, LA")

NO_coord = []
NO_coord.append(location.latitude)
NO_coord.append(location.longitude)

NO_coord

[29.9499323, -90.0701156]

In [6]:
forecast = forecastio.load_forecast(api_key, NO_coord[0], NO_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Rain on Wednesday through next Friday, with high temperatures rising to 71°F on Thursday.
rain
62.85


In [7]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28']

In [8]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [9]:
NO_table = pd.merge(df, NewOrleans_airbnb, on="Date")
NO_table["City"] = "New Orleans"
NO_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-21,55.64,42.27,0.31,0.0001,rain,97,802,1016,567,283
1,2018-12-22,63.62,56.28,0.05,0.0003,rain,70,606,849,467,251
2,2018-12-23,69.35,49.76,0.12,0.0010,rain,82,845,701,438,207
3,2018-12-24,60.00,51.52,0.07,0.0001,rain,96,833,752,431,235
4,2018-12-25,62.85,59.38,0.17,0.0017,rain,97,833,729,441,248
5,2018-12-26,66.68,64.15,0.89,0.0242,rain,95,796,757,402,204
6,2018-12-27,71.07,60.19,0.99,0.0878,rain,67,624,622,371,221
7,2018-12-28,66.78,62.00,0.46,0.0074,rain,32,229,302,369,274
